# Action Recognition @ UCF101  
**Due date: 11:59 pm on Nov. 19, 2019 (Tuesday)**

## Description
---
In this homework, you will be doing action recognition using Recurrent Neural Network (RNN), (Long-Short Term Memory) LSTM in particular. You will be given a dataset called UCF101, which consists of 101 different actions/classes and for each action, there will be 145 samples. We tagged each sample into either training or testing. Each sample is supposed to be a short video, but we sampled 25 frames from each videos to reduce the amount of data. Consequently, a training sample is an image tuple that forms a 3D volume with one dimension encoding *temporal correlation* between frames and a label indicating what action it is.

To tackle this problem, we aim to build a neural network that can not only capture spatial information of each frame but also temporal information between frames. Fortunately, you don't have to do this on your own. RNN — a type of neural network designed to deal with time-series data — is right here for you to use. In particular, you will be using LSTM for this task.

Instead of training an end-to-end neural network from scratch whose computation is prohibitively expensive, we divide this into two steps: feature extraction and modelling. Below are the things you need to implement for this homework:
- **{35 pts} Feature extraction**. Use any of the [pre-trained models](https://pytorch.org/docs/stable/torchvision/models.html) to extract features from each frame. Specifically, we recommend not to use the activations of the last layer as the features tend to be task specific towards the end of the network. 
    **hints**: 
    - A good starting point would be to use a pre-trained VGG16 network, we suggest first fully connected layer `torchvision.models.vgg16` (4096 dim) as features of each video frame. This will result into a 4096x25 matrix for each video. 
    - Normalize your images using `torchvision.transforms` 
    ```
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    prep = transforms.Compose([ transforms.ToTensor(), normalize ])
    prep(img)
    The mean and std. mentioned above is specific to Imagenet data
    
    ```
    More details of image preprocessing in PyTorch can be found at http://pytorch.org/tutorials/beginner/data_loading_tutorial.html
    
- **{35 pts} Modelling**. With the extracted features, build an LSTM network which takes a **dx25** sample as input (where **d** is the dimension of the extracted feature for each frame), and outputs the action label of that sample.
- **{20 pts} Evaluation**. After training your network, you need to evaluate your model with the testing data by computing the prediction accuracy **(5 points)**. The baseline test accuracy for this data is 75%, and **10 points** out of 20 is for achieving test accuracy greater than the baseline. Moreover, you need to compare **(5 points)** the result of your network with that of support vector machine (SVM) (stacking the **dx25** feature matrix to a long vector and train a SVM).
- **{10 pts} Report**. Details regarding the report can be found in the submission section below.

Notice that the size of the raw images is 256x340, whereas your pre-trained model might take **nxn** images as inputs. To solve this problem, instead of resizing the images which unfavorably changes the spatial ratio, we take a better solution: Cropping five **nxn** images, one at the image center and four at the corners and compute the **d**-dim features for each of them, and average these five **d**-dim feature to get a final feature representation for the raw image.
For example, VGG takes 224x224 images as inputs, so we take the five 224x224 croppings of the image, compute 4096-dim VGG features for each of them, and then take the mean of these five 4096-dim vectors to be the representation of the image.

In order to save you computational time, you need to do the classification task only for **the first 25** classes of the whole dataset. The same applies to those who have access to GPUs. **Bonus 10 points for running and reporting on the entire 101 classes.**


## Dataset
Download **dataset** at [UCF101](http://vision.cs.stonybrook.edu/~yangwang/public/UCF101_images.tar)(Image data for each video) and the **annos folder** which has the video labels and the label to class name mapping is included in the assignment folder uploaded. 


UCF101 dataset contains 101 actions and 13,320 videos in total.  

+ `annos/actions.txt`  
  + lists all the actions (`ApplyEyeMakeup`, .., `YoYo`)   
  
+ `annots/videos_labels_subsets.txt`  
  + lists all the videos (`v_000001`, .., `v_013320`)  
  + labels (`1`, .., `101`)  
  + subsets (`1` for train, `2` for test)  

+ `images/`  
  + each folder represents a video
  + the video/folder name to class mapping can be found using `annots/videos_labels_subsets.txt`, for e.g. `v_000001` belongs to class 1 i.e. `ApplyEyeMakeup`
  + each video folder contains 25 frames  



## Some Tutorials
- Good materials for understanding RNN and LSTM
    - http://blog.echen.me
    - http://karpathy.github.io/2015/05/21/rnn-effectiveness/
    - http://colah.github.io/posts/2015-08-Understanding-LSTMs/
- Implementing RNN and LSTM with PyTorch
    - [LSTM with PyTorch](http://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html#sphx-glr-beginner-nlp-sequence-models-tutorial-py)
    - [RNN with PyTorch](http://pytorch.org/tutorials/intermediate/char_rnn_classification_tutorial.html)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd '/content/drive/My Drive/HW5_CV'


/content/drive/My Drive/HW5_CV


In [3]:
cd 'images'

/content/drive/My Drive/HW5_CV/images


---
---
## **Problem 1.** Feature extraction

In [0]:
# \*write your codes for feature extraction (You can use multiple cells, this is just a place holder)
# \*write your codes for feature extraction (You can use multiple cells, this is just a place holder)
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import cv2
import numpy as np
import scipy.io
import numpy as np
from random import shuffle
import copy
from sklearn.metrics import accuracy_score 
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score
import pickle

In [0]:
# \*write your codes for feature extraction (You can use multiple cells, this is just a place holder)
train_features=[]
test_features=[]
train_labels=[]
test_labels=[]
total_data=[]
shuffle_data=[]
file_path='/content/drive/My Drive/CV_HW5/annos/videos_labels_subsets.txt'

with open(file_path,'r') as fp:
  line=fp.read()
  line_list=line.split("\n")

  for l in line_list:
    l1=l.split("\t")
    #print(l1)
    total_data.append(l1)
#print(total_data)

for i in range(0,3360):
  shuffle_data.append(total_data[i])
shuffle(shuffle_data)

for i in range(0,3360):
  t=total_data[i][2]
  #print (t)
  if t=='1':
    train_features.append(shuffle_data[i][0])
    train_labels.append(shuffle_data[i][1])
  elif t=='2':
    test_features.append(shuffle_data[i][0])
    test_labels.appendshuffle_data[i][1])

#len(test_features)
#len(train_features)

In [6]:
import torchvision.models as models
import torch.optim as opti

import torch.nn as nn

pre_train = models.vgg16(pretrained=True)

#n=pre_train.classifier[6].in_features
feat = list(pre_train.classifier.children())[:2]

pre_train.classifier = nn.Sequential(*feat)

opt = opti.SGD(pre_train.parameters(), lr=0.001, momentum=0.9)
crit = nn.CrossEntropyLoss()


print (pre_train)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [0]:

from PIL import Image
import glob
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
from scipy import io
import pickle
  #image_list = []
pre_train.cuda()
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
prep = transforms.Compose([ transforms.ToTensor(), normalize ])
mean_list=[]
c=0
feat_1=[]
feat_stack=[]
torch.no_grad()
for names in train_features[0:500]:
  full_path = "/content/drive/My Drive/HW5_CV/images/" + names + "/*"
  images_path = [name for name in glob.glob(full_path)]
  print (len(images_path))

  with torch.no_grad():    
    for image_path in images_path:
      #print (image_path)
      img = cv2.imread(image_path)
            
      #for filename in glob.glob(train_features[i]+"/*"):
      #paths=[name for name in glob.glob(train_features[i]+"/*")]
      #print (len(paths))
      #for path in paths:
        
       # image=cv2.imread(path)
        #image_list.append(image)
      #print (len(image_list))
      #for img in image_list:
      img1=img[16:240,58:282]
      img2=img[0:224,0:224]
      img3=img[0:224,116:340]
      img4=img[32:256,0:224]
      img5=img[32:256,116:340]

      img1=prep(img1)
      img1=img1.unsqueeze(0)
      img2=prep(img2)
      img2=img2.unsqueeze(0)
      img3=prep(img3)
      img3=img3.unsqueeze(0)
      img4=prep(img4)
      img4=img4.unsqueeze(0)
      img5=prep(img5)
      img5=img5.unsqueeze(0)
              
      img1=img1.cuda()
      feat1=pre_train(img1)
      img2=img2.cuda()
      feat2=pre_train(img2)
      img3=img3.cuda()
      feat3=pre_train(img3)
      img4=img4.cuda()
      feat4=pre_train(img4)
      img5=img5.cuda()
      feat5=pre_train(img5)
      #inp=[img1,img2,img3,img4,img5] 
      #inp=torch.stack(inp)
      #out=pre_train(inp)  
      #mean=out.mean(0)
      mean=(feat1+feat2+feat3+feat4+feat5)/5
                  #mean = torch.mean(my_list)
      #io.savemat('features_final.mat',{"features":mean})
      #feat.append(mean)
      feat_1.append(mean)
    #a=torch.stack(feat_1)
      #io.savemat('features_final_test.mat',{"features":feat_1})
    #feat_stack.append(a)
  c=c+1
  print (c)


In [0]:
import pickle
pick=open('train_1.pkl','wb')
pickle.dump(feat_1,pick)
pick.close()

In [0]:
del feat_1

In [0]:
f=open('train_1.pkl','rb')
data1=pickle.load(f)

In [0]:

from PIL import Image
import glob
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
from scipy import io
import pickle
  #image_list = []
pre_train.cuda()
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
prep = transforms.Compose([ transforms.ToTensor(), normalize ])
mean_list=[]
c=0
feat_1=[]
feat_stack=[]
torch.no_grad()
for names in train_features[500:1000]:
  full_path = "/content/drive/My Drive/HW5_CV/images/" + names + "/*"
  images_path = [name for name in glob.glob(full_path)]
  #print (images_path)

  with torch.no_grad():    
    for image_path in images_path:
      #print (image_path)
      img = cv2.imread(image_path)
            
      #for filename in glob.glob(train_features[i]+"/*"):
      #paths=[name for name in glob.glob(train_features[i]+"/*")]
      #print (len(paths))
      #for path in paths:
        
       # image=cv2.imread(path)
        #image_list.append(image)
      #print (len(image_list))
      #for img in image_list:
      img1=img[16:240,58:282]
      img2=img[0:224,0:224]
      img3=img[0:224,116:340]
      img4=img[32:256,0:224]
      img5=img[32:256,116:340]

      img1=prep(img1)
      img1=img1.unsqueeze(0)
      img2=prep(img2)
      img2=img2.unsqueeze(0)
      img3=prep(img3)
      img3=img3.unsqueeze(0)
      img4=prep(img4)
      img4=img4.unsqueeze(0)
      img5=prep(img5)
      img5=img5.unsqueeze(0)
              
      img1=img1.cuda()
      feat1=pre_train(img1)
      img2=img2.cuda()
      feat2=pre_train(img2)
      img3=img3.cuda()
      feat3=pre_train(img3)
      img4=img4.cuda()
      feat4=pre_train(img4)
      img5=img5.cuda()
      feat5=pre_train(img5)
      #inp=[img1,img2,img3,img4,img5] 
      #inp=torch.stack(inp)
      #out=pre_train(inp)  
      #mean=out.mean(0)
      mean=(feat1+feat2+feat3+feat4+feat5)/5
                  #mean = torch.mean(my_list)
      #io.savemat('features_final.mat',{"features":mean})
      #feat.append(mean)
      feat_1.append(mean)
    #a=torch.stack(feat_1)
      #io.savemat('features_final_test.mat',{"features":feat_1})
    #feat_stack.append(a)
  c=c+1
  print (c)


In [0]:
pick=open('train_2.pkl','wb')
pickle.dump(feat_1,pick)
pick.close()

In [0]:
del feat_1

In [0]:

from PIL import Image
import glob
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
from scipy import io
import pickle
  #image_list = []
pre_train.cuda()
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
prep = transforms.Compose([ transforms.ToTensor(), normalize ])
mean_list=[]
c=0
feat_1=[]
feat_stack=[]
torch.no_grad()
for names in train_features[1000:1500]:
  full_path = "/content/drive/My Drive/HW5_CV/images/" + names + "/*"
  images_path = [name for name in glob.glob(full_path)]
  #print (images_path)

  with torch.no_grad():    
    for image_path in images_path:
      #print (image_path)
      img = cv2.imread(image_path)
            
      #for filename in glob.glob(train_features[i]+"/*"):
      #paths=[name for name in glob.glob(train_features[i]+"/*")]
      #print (len(paths))
      #for path in paths:
        
       # image=cv2.imread(path)
        #image_list.append(image)
      #print (len(image_list))
      #for img in image_list:
      img1=img[16:240,58:282]
      img2=img[0:224,0:224]
      img3=img[0:224,116:340]
      img4=img[32:256,0:224]
      img5=img[32:256,116:340]

      img1=prep(img1)
      img1=img1.unsqueeze(0)
      img2=prep(img2)
      img2=img2.unsqueeze(0)
      img3=prep(img3)
      img3=img3.unsqueeze(0)
      img4=prep(img4)
      img4=img4.unsqueeze(0)
      img5=prep(img5)
      img5=img5.unsqueeze(0)
              
      img1=img1.cuda()
      feat1=pre_train(img1)
      img2=img2.cuda()
      feat2=pre_train(img2)
      img3=img3.cuda()
      feat3=pre_train(img3)
      img4=img4.cuda()
      feat4=pre_train(img4)
      img5=img5.cuda()
      feat5=pre_train(img5)
      #inp=[img1,img2,img3,img4,img5] 
      #inp=torch.stack(inp)
      #out=pre_train(inp)  
      #mean=out.mean(0)
      mean=(feat1+feat2+feat3+feat4+feat5)/5
                  #mean = torch.mean(my_list)
      #io.savemat('features_final.mat',{"features":mean})
      #feat.append(mean)
      feat_1.append(mean)
    #a=torch.stack(feat_1)
      #io.savemat('features_final_test.mat',{"features":feat_1})
    #feat_stack.append(a)
  c=c+1
  print (c)


In [0]:
pick=open('train_3.pkl','wb')
pickle.dump(feat_1,pick)
pick.close()

In [0]:
del feat_1

In [0]:

from PIL import Image
import glob
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
from scipy import io
import pickle
  #image_list = []
pre_train.cuda()
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
prep = transforms.Compose([ transforms.ToTensor(), normalize ])
mean_list=[]
c=0
feat_1=[]
feat_stack=[]
torch.no_grad()
for names in train_features[1500:2000]:
  full_path = "/content/drive/My Drive/HW5_CV/images/" + names + "/*"
  images_path = [name for name in glob.glob(full_path)]
  #print (images_path)

  with torch.no_grad():    
    for image_path in images_path:
      #print (image_path)
      img = cv2.imread(image_path)
            
      #for filename in glob.glob(train_features[i]+"/*"):
      #paths=[name for name in glob.glob(train_features[i]+"/*")]
      #print (len(paths))
      #for path in paths:
        
       # image=cv2.imread(path)
        #image_list.append(image)
      #print (len(image_list))
      #for img in image_list:
      img1=img[16:240,58:282]
      img2=img[0:224,0:224]
      img3=img[0:224,116:340]
      img4=img[32:256,0:224]
      img5=img[32:256,116:340]

      img1=prep(img1)
      img1=img1.unsqueeze(0)
      img2=prep(img2)
      img2=img2.unsqueeze(0)
      img3=prep(img3)
      img3=img3.unsqueeze(0)
      img4=prep(img4)
      img4=img4.unsqueeze(0)
      img5=prep(img5)
      img5=img5.unsqueeze(0)
              
      img1=img1.cuda()
      feat1=pre_train(img1)
      img2=img2.cuda()
      feat2=pre_train(img2)
      img3=img3.cuda()
      feat3=pre_train(img3)
      img4=img4.cuda()
      feat4=pre_train(img4)
      img5=img5.cuda()
      feat5=pre_train(img5)
      #inp=[img1,img2,img3,img4,img5] 
      #inp=torch.stack(inp)
      #out=pre_train(inp)  
      #mean=out.mean(0)
      mean=(feat1+feat2+feat3+feat4+feat5)/5
                  #mean = torch.mean(my_list)
      #io.savemat('features_final.mat',{"features":mean})
      #feat.append(mean)
      feat_1.append(mean)
    #a=torch.stack(feat_1)
      #io.savemat('features_final_test.mat',{"features":feat_1})
    #feat_stack.append(a)
  c=c+1
  print (c)


In [0]:
pick=open('train_4.pkl','wb')
pickle.dump(feat_1,pick)
pick.close()

In [0]:
del feat_1

In [0]:

from PIL import Image
import glob
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
from scipy import io
import pickle
  #image_list = []
pre_train.cuda()
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
prep = transforms.Compose([ transforms.ToTensor(), normalize ])
mean_list=[]
c=0
feat_1=[]
feat_stack=[]
torch.no_grad()
for names in train_features[2000:]:
  full_path = "/content/drive/My Drive/HW5_CV/images/" + names + "/*"
  images_path = [name for name in glob.glob(full_path)]
  #print (images_path)

  with torch.no_grad():    
    for image_path in images_path:
      #print (image_path)
      img = cv2.imread(image_path)
            
      #for filename in glob.glob(train_features[i]+"/*"):
      #paths=[name for name in glob.glob(train_features[i]+"/*")]
      #print (len(paths))
      #for path in paths:
        
       # image=cv2.imread(path)
        #image_list.append(image)
      #print (len(image_list))
      #for img in image_list:
      img1=img[16:240,58:282]
      img2=img[0:224,0:224]
      img3=img[0:224,116:340]
      img4=img[32:256,0:224]
      img5=img[32:256,116:340]

      img1=prep(img1)
      img1=img1.unsqueeze(0)
      img2=prep(img2)
      img2=img2.unsqueeze(0)
      img3=prep(img3)
      img3=img3.unsqueeze(0)
      img4=prep(img4)
      img4=img4.unsqueeze(0)
      img5=prep(img5)
      img5=img5.unsqueeze(0)
              
      img1=img1.cuda()
      feat1=pre_train(img1)
      img2=img2.cuda()
      feat2=pre_train(img2)
      img3=img3.cuda()
      feat3=pre_train(img3)
      img4=img4.cuda()
      feat4=pre_train(img4)
      img5=img5.cuda()
      feat5=pre_train(img5)
      #inp=[img1,img2,img3,img4,img5] 
      #inp=torch.stack(inp)
      #out=pre_train(inp)  
      #mean=out.mean(0)
      mean=(feat1+feat2+feat3+feat4+feat5)/5
                  #mean = torch.mean(my_list)
      #io.savemat('features_final.mat',{"features":mean})
      #feat.append(mean)
      feat_1.append(mean)
    #a=torch.stack(feat_1)
      #io.savemat('features_final_test.mat',{"features":feat_1})
    #feat_stack.append(a)
  c=c+1
  print (c)


In [0]:
pick=open('train_5.pkl','wb')
pickle.dump(feat_1,pick)
pick.close()

In [0]:
del feat_1

In [0]:

from PIL import Image
import glob
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
from scipy import io
import pickle
  #image_list = []
pre_train.cuda()
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
prep = transforms.Compose([ transforms.ToTensor(), normalize ])
mean_list=[]
c=0
feat_1=[]
feat_stack=[]
torch.no_grad()
for names in test_features:
  full_path = "/content/drive/My Drive/HW5_CV/images/" + names + "/*"
  images_path = [name for name in glob.glob(full_path)]
  #print (images_path)

  with torch.no_grad():    
    for image_path in images_path:
      #print (image_path)
      img = cv2.imread(image_path)
            
      #for filename in glob.glob(train_features[i]+"/*"):
      #paths=[name for name in glob.glob(train_features[i]+"/*")]
      #print (len(paths))
      #for path in paths:
        
       # image=cv2.imread(path)
        #image_list.append(image)
      #print (len(image_list))
      #for img in image_list:
      img1=img[16:240,58:282]
      img2=img[0:224,0:224]
      img3=img[0:224,116:340]
      img4=img[32:256,0:224]
      img5=img[32:256,116:340]

      img1=prep(img1)
      img1=img1.unsqueeze(0)
      img2=prep(img2)
      img2=img2.unsqueeze(0)
      img3=prep(img3)
      img3=img3.unsqueeze(0)
      img4=prep(img4)
      img4=img4.unsqueeze(0)
      img5=prep(img5)
      img5=img5.unsqueeze(0)
              
      img1=img1.cuda()
      feat1=pre_train(img1)
      img2=img2.cuda()
      feat2=pre_train(img2)
      img3=img3.cuda()
      feat3=pre_train(img3)
      img4=img4.cuda()
      feat4=pre_train(img4)
      img5=img5.cuda()
      feat5=pre_train(img5)
      #inp=[img1,img2,img3,img4,img5] 
      #inp=torch.stack(inp)
      #out=pre_train(inp)  
      #mean=out.mean(0)
      mean=(feat1+feat2+feat3+feat4+feat5)/5
                  #mean = torch.mean(my_list)
      #io.savemat('features_final.mat',{"features":mean})
      #feat.append(mean)
      feat_1.append(mean)
    #a=torch.stack(feat_1)
      #io.savemat('features_final_test.mat',{"features":feat_1})
    #feat_stack.append(a)
  c=c+1
  print (c)


In [0]:
pick=open('test.pkl','wb')
pickle.dump(feat_1,pick)
pick.close()

In [0]:
import pickle
f=open('train_1.pkl','rb')
d1=pickle.load(f)
f=open('train_2.pkl','rb')
d2=pickle.load(f)
f=open('train_3.pkl','rb')
d3=pickle.load(f)
f=open('train_4.pkl','rb')
d4=pickle.load(f)
f=open('train_5.pkl','rb')
d5=pickle.load(f)
train_data=d1+d2+d3+d4+d5

In [0]:
f=open('train_final.pkl','wb')
pickle.load(train_data,f)

In [0]:
f=open('test_final.pkl','wb')
pickle.load(test_data,f)

In [0]:
f=open('train_final.pkl','rb')
train_data=pickle.load(f)

In [0]:
f=open('test_final.pkl','rb')
test_data=pickle.load(f)

In [0]:
#SInce the shape of train_data after loading the pickle was [1,4096], converting it to [4,25,4096]
train1 = [t.view(-1) for t in train_data]
#print train1[0].shape
train2=[]
for i in range(0,len(train1),25):
  imd1=train1[i:i+25]
  imd2=torch.stack(imd1)
  train2.append(imd2)
#print train2[0].shape

train3=[]
for i in range(0,len(train2),4):
  imd1=train2[i:i+4]
  imd2=torch.stack(imd1)
  train3.append(imd2)
#print train3[0].shape

train4=[]
for i in range(len(train_data)):
  a,b,c=train_data[i].shape
  data=train_data[i].view(b,a,c)
  train4.append(data)
#print train4[0].shape


In [0]:
train_label1=[]
for i in range(len(train_labels)):
  a=train_labels[i].view(-1).long()
  a=a-torch.ones(a.size()).long()
  train_label1.append(a)

In [0]:
#SInce the shape of train_data after loading the pickle was [1,4096], converting it to [4,25,4096]
test1 = [t.view(-1) for t in test_data]
#print test1[0].shape
test2=[]
for i in range(0,len(test1),25):
  imd1=test1[i:i+25]
  imd2=torch.stack(imd1)
  test2.append(imd2)
#print test2[0].shape

test3=[]
for i in range(0,len(test2),4):
  imd1=test2[i:i+4]
  imd2=torch.stack(imd1)
  test3.append(imd2)
#print test3[0].shape

test4=[]
for i in range(len(test_data)):
  a,b,c=test_data[i].shape
  data=test_data[i].view(b,a,c)
  test4.append(data)
#print test4[0].shape


In [0]:
test_label1=[]
for i in range(len(test_labels)):
  a=test_labels[i].view(-1).long()
  a=a-torch.ones(a.size()).long()
  test_label1.append(a)

***
***
## **Problem 2.** Modelling

* ##### **Print the size of your training and test data**

In [87]:
# Don't hardcode the shape of train and test data
print('Shape of training data is :', )
print('Shape of test/validation data is :', )

Shape of training data is :
Shape of test/validation data is :


In [0]:
class LSTM(nn.Module):    
    def __init__(self, hidden_dim, num_lab):
        super(LSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(4096, hidden_dim)
        self.hiddenlabel = nn.Linear(hidden_dim, num_lab)   

    def forward(self, input_features):
        batch_size = input_features.size()[1]
        lstm_out, last_op = self.lstm(input_features)
        out = self.hiddenlabel(last_op[0].view(batch_size,-1))
        output_labels = F.log_softmax(out, dim=-1)
        return output_labels

In [80]:
import torch.optim as optim
import time
Hidden_dim = 256
Num_lab =25
t1=time.time()
model_lstm = LSTM(Hidden_dim,Num_lab)
print(model_lstm)
l_func = nn.NLLLoss()
opt = optim.SGD(model_lstm.parameters(), lr=0.01)

#Training
initial_loss = 0
for e in range(37):
    l_count = 0.0
    for i in range(len(train_data)):
        features = train_data1[i]
        labels = train_label1[i]
        model_lstm.zero_grad()
        outputs = model_lstm(features)
        loss = l_func(outputs, labels)
        loss.backward()
        opt.step()
        l_count += loss.item()
    print('Epoch: %d Loss: %.3f' %(e+1, l_count/len(train_data)))

print ("Training done in",(time.time()-t1)/60,"minutes")

LSTM(
  (lstm): LSTM(4096, 256)
  (hiddenlabel): Linear(in_features=256, out_features=25, bias=True)
)
Epoch: 1 Loss: 3.217
Epoch: 2 Loss: 3.125
Epoch: 3 Loss: 3.020
Epoch: 4 Loss: 2.894
Epoch: 5 Loss: 2.750
Epoch: 6 Loss: 2.586
Epoch: 7 Loss: 2.446
Epoch: 8 Loss: 2.309
Epoch: 9 Loss: 2.178
Epoch: 10 Loss: 2.054
Epoch: 11 Loss: 1.946
Epoch: 12 Loss: 1.811
Epoch: 13 Loss: 1.701
Epoch: 14 Loss: 1.611
Epoch: 15 Loss: 1.498
Epoch: 16 Loss: 1.423
Epoch: 17 Loss: 1.330
Epoch: 18 Loss: 1.241
Epoch: 19 Loss: 1.216
Epoch: 20 Loss: 1.066
Epoch: 21 Loss: 1.006
Epoch: 22 Loss: 0.969
Epoch: 23 Loss: 0.969
Epoch: 24 Loss: 0.961
Epoch: 25 Loss: 0.933
Epoch: 26 Loss: 0.844
Epoch: 27 Loss: 0.736
Epoch: 28 Loss: 0.631
Epoch: 29 Loss: 0.563
Epoch: 30 Loss: 0.484
Epoch: 31 Loss: 0.463
Epoch: 32 Loss: 0.365
Epoch: 33 Loss: 0.306
Epoch: 34 Loss: 0.240
Epoch: 35 Loss: 0.168
Epoch: 36 Loss: 0.111
Epoch: 37 Loss: 0.074
Training done in 15.106865974267324 minutes


In [81]:
with torch.no_grad():
  corr = 0
  t=0
  for i in range(len(train_data)):
      feats = train_data1[i]
      labels = train_label1[i]
      outputs = model_lstm(feats)
      _, pred = torch.max(outputs.data, 1)
      t += labels.size(0)
      corr += (pred == labels).sum().item()
      accuracy = (corr/t)*100
print("Accuracy on the train dataset is:", accuracy)

Accuracy on the train dataset is: 99.75093399750934


In [82]:
with torch.no_grad():
  corr = 0
  t=0
  for i in range(len(test_data)):
      feats = test_data1[i]
      labels = test_label1[i]
      outputs = model_lstm(feats)
      _, pred = torch.max(outputs.data, 1)
      t += labels.size(0)
      corr += (pred == labels).sum().item()
      accuracy_test = (corr/t)*100
print("Accuracy on the test dataset is:", accuracy_test)

Accuracy on the test dataset is: 79.81072555205047


In [76]:
import torch.optim as optim
import time
Hidden_dim = 512
Num_lab =25
t1=time.time()
model_lstm1 = LSTM(Hidden_dim,Num_lab)
print (model_lstm1)
l_func = nn.NLLLoss()
opt = optim.SGD(model_lstm1.parameters(), lr=0.01)

#Training
initial_loss = 0
for e in range(37):
    l_count = 0.0
    for i in range(len(train_data)):
        features = train_data1[i]
        labels = train_label1[i]
        model_lstm1.zero_grad()
        outputs = model_lstm1(features)
        loss = l_func(outputs, labels)
        loss.backward()
        opt.step()
        l_count += loss.item()
    print('Epoch: %d Loss: %.3f' %(e+1, l_count/len(train_data)))

print ("Training done in",(time.time()-t1)/60,"minutes")

LSTM(
  (lstm): LSTM(4096, 512)
  (hiddenlabel): Linear(in_features=512, out_features=25, bias=True)
)
Epoch: 1 Loss: 3.203
Epoch: 2 Loss: 3.084
Epoch: 3 Loss: 2.941
Epoch: 4 Loss: 2.775
Epoch: 5 Loss: 2.599
Epoch: 6 Loss: 2.417
Epoch: 7 Loss: 2.263
Epoch: 8 Loss: 2.110
Epoch: 9 Loss: 1.977
Epoch: 10 Loss: 1.877
Epoch: 11 Loss: 1.777
Epoch: 12 Loss: 1.671
Epoch: 13 Loss: 1.604
Epoch: 14 Loss: 1.540
Epoch: 15 Loss: 1.459
Epoch: 16 Loss: 1.344
Epoch: 17 Loss: 1.285
Epoch: 18 Loss: 1.201
Epoch: 19 Loss: 1.183
Epoch: 20 Loss: 1.143
Epoch: 21 Loss: 1.070
Epoch: 22 Loss: 1.005
Epoch: 23 Loss: 0.951
Epoch: 24 Loss: 0.900
Epoch: 25 Loss: 0.819
Epoch: 26 Loss: 0.853
Epoch: 27 Loss: 0.795
Epoch: 28 Loss: 0.696
Epoch: 29 Loss: 0.651
Epoch: 30 Loss: 0.583
Epoch: 31 Loss: 0.521
Epoch: 32 Loss: 0.452
Epoch: 33 Loss: 0.403
Epoch: 34 Loss: 0.348
Epoch: 35 Loss: 0.296
Epoch: 36 Loss: 0.261
Epoch: 37 Loss: 0.191
Training done in 36.734095601240796 minutes


In [77]:
with torch.no_grad():
  corr = 0
  t=0
  for i in range(len(train_data)):
      feats = train_data1[i]
      labels = train_label1[i]
      outputs = model_lstm1(feats)
      _, pred = torch.max(outputs.data, 1)
      t += labels.size(0)
      corr += (pred == labels).sum().item()
      accuracy_NEW = (corr/t)*100
print("Accuracy on the train dataset is:", accuracy_NEW)

Accuracy on the train dataset is: 96.13947696139476


In [78]:
with torch.no_grad():
  corr = 0
  t=0
  for i in range(len(test_data)):
      feats = test_data1[i]
      labels = test_label1[i]
      outputs = model_lstm1(feats)
      _, pred = torch.max(outputs.data, 1)
      t += labels.size(0)
      corr += (pred == labels).sum().item()
      accuracy_test_NEW = (corr/t)*100
print("Accuracy on the test dataset is:", accuracy_test_NEW)

Accuracy on the test dataset is: 78.44374342797056


---
---
## **Problem 3.** Evaluation

* ##### **Print the train and test accuracy of your model** 

In [84]:
# Don't hardcode the train and test accuracy
#For hidden dimension=256
print('Training accuracy is %2.3f :', accuracy )
print('Test accuracy is %2.3f :', accuracy_test)

Training accuracy is %2.3f : 99.75093399750934
Test accuracy is %2.3f : 79.81072555205047


In [85]:
# Don't hardcode the train and test accuracy
#For hidden dimension=512
print('Training accuracy is %2.3f :', accuracy_NEW )
print('Test accuracy is %2.3f :', accuracy_test_NEW)

Training accuracy is %2.3f : 96.13947696139476
Test accuracy is %2.3f : 78.44374342797056


* ##### **Print the train and test and test accuracy of SVM** 

In [0]:
# Don't hardcode the train and test accuracy
print('Training accuracy is %2.3f :' %(100.00) )
print('Test accuracy is %2.3f :' %(100.00) )

Training accuracy is 100.000 :
Test accuracy is 100.000 :


## **Problem 4.** Report

## **Bonus**


* ##### **Print the size of your training and test data**

In [0]:
# Don't hardcode the shape of train and test data
print('Shape of training data is :', )
print('Shape of test/validation data is :', )

* ##### **Modelling and evaluation**

In [0]:
#Write your code for modelling and evaluation

## Submission
---
**Runnable source code in ipynb file and a pdf report are required**.

The report should be of 3 to 4 pages describing what you have done and learned in this homework and report performance of your model. If you have tried multiple methods, please compare your results. If you are using any external code, please cite it in your report. Note that this homework is designed to help you explore and get familiar with the techniques. The final grading will be largely based on your prediction accuracy and the different methods you tried (different architectures and parameters).

Please indicate clearly in your report what model you have tried, what techniques you applied to improve the performance and report their accuracies. The report should be concise and include the highlights of your efforts.
The naming convention for report is **Surname_Givenname_SBUID_report*.pdf**

When submitting your .zip file through blackboard, please
-- name your .zip file as **Surname_Givenname_SBUID_hw*.zip**.

This zip file should include:
```
Surname_Givenname_SBUID_hw*
        |---Surname_Givenname_SBUID_hw*.ipynb
        |---Surname_Givenname_SBUID_hw*.pdf
        |---Surname_Givenname_SBUID_report*.pdf
```

For instance, student Michael Jordan should submit a zip file named "Jordan_Michael_111134567_hw5.zip" for homework5 in this structure:
```
Jordan_Michael_111134567_hw5
        |---Jordan_Michael_111134567_hw5.ipynb
        |---Jordan_Michael_111134567_hw5.pdf
        |---Jordan_Michael_111134567_report*.pdf
```

The **Surname_Givenname_SBUID_hw*.pdf** should include a **google shared link**. To generate the **google shared link**, first create a folder named **Surname_Givenname_SBUID_hw*** in your Google Drive with your Stony Brook account. 

Then right click this folder, click ***Get shareable link***, in the People textfield, enter two TA's emails: ***bo.cao.1@stonybrook.edu*** and ***sayontan.ghosh@stonybrook.edu***. Make sure that TAs who have the link **can edit**, ***not just*** **can view**, and also **uncheck** the **Notify people** box.

Colab has a good feature of version control, you should take advantage of this to save your work properly. However, the timestamp of the submission made in blackboard is the only one that we consider for grading. To be more specific, we will only grade the version of your code right before the timestamp of the submission made in blackboard. 

You are encouraged to post and answer questions on Piazza. Based on the amount of email that we have received in past years, there might be dealys in replying to personal emails. Please ask questions on Piazza and send emails only for personal issues.

Be aware that your code will undergo plagiarism check both vertically and horizontally. Please do your own work.